In [9]:
import numpy as np
import pandas as pd
import sidetable
import matplotlib.pyplot as plt

import statsmodels.api as sma
from scipy import stats
import stumpy
import pmdarima as pmd

from tqdm import tqdm_notebook
from collections import Counter
import seaborn as sns

import datetime as dt
import gc
import os
import warnings

%matplotlib inline

warnings.filterwarnings(action='ignore')
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [2]:
# data = pd.read_json('https://www.fema.gov/api/open/v1/FimaNfipPolicies')

data = pd.read_json('floodins.json')

In [3]:
data.head()

agricultureStructureIndicator  baseFloodElevation  \
0                          False                 NaN   
1                          False                 NaN   
2                          False                 NaN   
3                          False                 NaN   
4                          False                 NaN   

   basementEnclosureCrawlspace   censusTract  cancellationDateOfFloodPolicy  \
0                          0.0  2.902100e+10                            NaN   
1                          1.0  2.902100e+10                            NaN   
2                          NaN  2.902395e+10                            NaN   
3                          2.0  2.902100e+10                            NaN   
4                          NaN  2.902100e+10                            NaN   

  condominiumIndicator  construction countyCode  crsClassCode  \
0                    N         False      29021           NaN   
1                    N         False      29021           NaN   
2                    N         False      29023           NaN   
3                    N         False      29021           NaN   
4                    N         False      29021           NaN   

  deductibleAmountInBuildingCoverage deductibleAmountInContentsCoverage  \
0                                  2                                      
1                                  F                                  F   
2                                  0                                      
3                                  1                                  0   
4                                  F                                  F   

   elevationBuildingIndicator  elevationCertificateIndicator  \
0                       False                            NaN   
1                       False                            NaN   
2                        True                            2.0   
3                       False                            NaN   
4                       False                            NaN   

   elevationDifference  federalPolicyFee floodZone  hfiaaSurcharge  \
0                  NaN                50         A              25   
1                  NaN                25         B             250   
2                  NaN                35        AH               0   
3                  NaN                35       A04               0   
4                  NaN                25         B             250   

   houseOfWorshipIndicator  latitude  longitude  locationOfContents  \
0                    False      39.7      -94.7                 NaN   
1                    False      39.7      -94.9                 2.0   
2                    False      36.5      -90.4                 NaN   
3                    False      39.8      -94.8                 NaN   
4                    False      39.8      -94.9                 3.0   

   lowestAdjacentGrade  lowestFloorElevation  nonProfitIndicator  \
0                  NaN                   NaN               False   
1                  NaN                   NaN               False   
2                  NaN                   NaN               False   
3                  NaN                   NaN               False   
4                  NaN                   NaN               False   

   numberOfFloorsInTheInsuredBuilding  obstructionType  occupancyType  \
0                                 1.0              NaN              1   
1                                 2.0              NaN              1   
2                                 5.0             10.0              1   
3                                 2.0              NaN              1   
4                                 1.0              NaN              1   

   originalConstructionDate            originalNBDate  policyCost  \
0  2045-05-15T04:00:00.000Z  2004-10-16T04:00:00.000Z         725   
1  1974-03-01T04:00:00.000Z  2000-01-01T05:00:00.000Z         692   
2  1999-01-01T05:00:00.000Z  2008-03-06T05:00:00.000Z        

In [4]:
data.describe()

baseFloodElevation  basementEnclosureCrawlspace   censusTract  \
count          232.000000                   490.000000  9.820000e+02   
mean          1144.237069                     1.489796  2.949724e+10   
std           1666.202793                     0.922840  7.335365e+08   
min              0.000000                     0.000000  1.715795e+10   
25%            309.000000                     1.000000  2.909502e+10   
50%            457.000000                     2.000000  2.918331e+10   
75%            972.000000                     2.000000  3.001796e+10   
max           9990.000000                     4.000000  3.117705e+10   

       cancellationDateOfFloodPolicy  crsClassCode  \
count                            0.0    427.000000   
mean                             NaN      7.798595   
std                              NaN      0.820339   
min                              NaN      5.000000   
25%                              NaN      7.000000   
50%                              NaN      8.000000   
75%                              NaN      8.000000   
max                              NaN     10.000000   

       elevationCertificateIndicator  elevationDifference  federalPolicyFee  \
count                     169.000000           234.000000       1000.000000   
mean                        2.005917             1.188034         38.100000   
std                         0.984989             2.626807         10.959841   
min                         1.000000            -8.000000          0.000000   
25%                         1.000000             0.000000         25.000000   
50%                         2.000000             1.000000         40.000000   
75%                         3.000000             2.000000         50.000000   
max                         4.000000            14.000000         50.000000   

       hfiaaSurcharge     latitude    longitude  locationOfContents  \
count     1000.000000  1000.000000  1000.000000          499.000000   
mean        51.350000    40.540300   -96.706700            2.901804   
std         93.705433     3.838356     8.289028            1.156602   
min          0.000000    36.000000  -115.600000            1.000000   
25%          0.000000    38.100000  -100.725000            2.000000   
50%          0.000000    39.000000   -94.300000            3.000000   
75%         25.000000    41.900000   -90.400000            3.000000   
max        250.000000    48.600000   -89.200000            7.000000   

       lowestAdjacentGrade  lowestFloorElevation  \
count           220.000000            233.000000   
mean            951.227273           1140.651502   
std            1147.101268           1664.356351   
min               0.000000             -6.800000   
25%             305.000000            309.500000   
50%             447.000000            459.900000   
75%             869.000000            968.300000   
max            5331.000000           9992.000000   

       numberOfFloorsInTheInsuredBuilding  obstructionType  occupancyType  \
count                          999.000000       206.000000    1000.000000   
mean                             1.868869        33.587379       1.917000   
std                              0.900802        22.969750       1.804818   
min                              1.000000        10.000000       1.000000   
25%                              1.000000        10.000000       1.000000   
50%                              2.000000        50.000000       1.000000   
75%                              2.000000        50.000000       1.000000   
max                              5.000000        80.000000       6.000000   

        policyCost  policyCount  policyTermIndicator  reportedZipCode  \
count   1000.00000       1000.0          1000.000000      1000.000000   
mean    1091.25300          1.0             1.002000     63205.695000   
std     1577.88322          0.0             0.063246      2487.703691   
min       46.00000          1.0             1.000000     59011.0

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 46 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   agricultureStructureIndicator       1000 non-null   bool   
 1   baseFloodElevation                  232 non-null    float64
 2   basementEnclosureCrawlspace         490 non-null    float64
 3   censusTract                         982 non-null    float64
 4   cancellationDateOfFloodPolicy       0 non-null      float64
 5   condominiumIndicator                1000 non-null   object 
 6   construction                        1000 non-null   bool   
 7   countyCode                          1000 non-null   object 
 8   crsClassCode                        427 non-null    float64
 9   deductibleAmountInBuildingCoverage  1000 non-null   object 
 10  deductibleAmountInContentsCoverage  1000 non-null   object 
 11  elevationBuildingIndicator          1000 non

In [15]:
# data.stb.freq(['reportedCity']).sort_values(by='reportedCity').tail(100)

data[data.reportedCity.str.contains('PIT')]

Empty DataFrame
Columns: [agricultureStructureIndicator, baseFloodElevation, basementEnclosureCrawlspace, censusTract, cancellationDateOfFloodPolicy, condominiumIndicator, construction, countyCode, crsClassCode, deductibleAmountInBuildingCoverage, deductibleAmountInContentsCoverage, elevationBuildingIndicator, elevationCertificateIndicator, elevationDifference, federalPolicyFee, floodZone, hfiaaSurcharge, houseOfWorshipIndicator, latitude, longitude, locationOfContents, lowestAdjacentGrade, lowestFloorElevation, nonProfitIndicator, numberOfFloorsInTheInsuredBuilding, obstructionType, occupancyType, originalConstructionDate, originalNBDate, policyCost, policyCount, policyEffectiveDate, policyTerminationDate, policyTermIndicator, postFIRMConstructionIndicator, primaryResidenceIndicator, propertyState, reportedZipCode, rateMethod, regularEmergencyProgramIndicator, reportedCity, smallBusinessIndicatorBuilding, totalBuildingInsuranceCoverage, totalContentsInsuranceCoverage, totalInsurancePremiumOfThePolicy, id]
Index: []

In [19]:
data[data.countyCode.str.contains('152')]

Empty DataFrame
Columns: [agricultureStructureIndicator, baseFloodElevation, basementEnclosureCrawlspace, censusTract, cancellationDateOfFloodPolicy, condominiumIndicator, construction, countyCode, crsClassCode, deductibleAmountInBuildingCoverage, deductibleAmountInContentsCoverage, elevationBuildingIndicator, elevationCertificateIndicator, elevationDifference, federalPolicyFee, floodZone, hfiaaSurcharge, houseOfWorshipIndicator, latitude, longitude, locationOfContents, lowestAdjacentGrade, lowestFloorElevation, nonProfitIndicator, numberOfFloorsInTheInsuredBuilding, obstructionType, occupancyType, originalConstructionDate, originalNBDate, policyCost, policyCount, policyEffectiveDate, policyTerminationDate, policyTermIndicator, postFIRMConstructionIndicator, primaryResidenceIndicator, propertyState, reportedZipCode, rateMethod, regularEmergencyProgramIndicator, reportedCity, smallBusinessIndicatorBuilding, totalBuildingInsuranceCoverage, totalContentsInsuranceCoverage, totalInsurancePremiumOfThePolicy, id]
Index: []